In [32]:
import re
import numpy as np
import pandas as pd
import time
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

In [3]:
films = []
for p in Path("C:/Users/forem/Code/text-as-data/people/Foreman/Adventure").glob('*.txt'):
    with open(p, mode="r", encoding="utf-8") as f:
        film = f.read()
        films.append(film)

In [33]:
tokenized_films = []
for film in films:
    tokenized_film = word_tokenize(film)
    tokenized_films.append(tokenized_film)

# for i, tokenized_film in enumerate(tokenized_films):
#     print(f"Film {i+1}:")
#     for sentence in tokenized_film:
#         print(sentence)

In [5]:
print(f"Number of tokenized films: {len(tokenized_films)}")

Number of tokenized films: 166


In [6]:
# for sentence in tokenized_films[0][:20]:
#     words = re.sub("[^a-zA-Z']"," ", sentence).split()
#     print(" ".join(words))

HOURS Written by Simon Beaufoy Danny Boyle EXT
CROWD SCENES
VARIOUS
A massive crowd it could be a sports stadium a u farewell show or new year's eve on copacabana beach but whatever it is there are thousands and thousands of us
A mexican wave erupts success celebration with so many involved it's impossible to pick out anyone individually
Critical mass cyclists easter crowds at st Peter's basilica nyc marathon flash mobbers doing the silent disco at london's victoria station india's kumbh mela macy's thanksgiving day parade raves subway parties the daytona


Gradually the screen splits into and then though at times there appears to be no division at all
EXT
FREEWAY
NIGHT
An overhead shot of a crammed freeway gives way to a single vehicle a Toyota Tacoma red and white with a topper
CUT TO INT
ARON'S TRUCK
NIGHT
Cut inside as Aron Ralston cuts off the freeway
TRIPTYCH
OPENING TITLES ARE A SERIES OF TRIPTYCHS FEATURING ALL THE TITLE CARDS EXCEPT THE MAIN ONE
THEY BLEND OVERLAP AND ARE INTE

In [31]:
# Assuming tokenized_films is a list of tokenized text files
# Define the chunk sizes
chunk1_size = 0.25
chunk2_size = 0.50
chunk3_size = 0.25

# Initialize a list to store the chunks for each file
chunked_tokenized_films = []

for film in tokenized_films:
    # Calculate the sizes of each chunk based on the length of the file
    total_length = len(film)
    chunk1_end = int(total_length * chunk1_size)
    chunk2_end = int(total_length * (chunk1_size + chunk2_size))

    # Divide the file into three chunks
    chunk1 = film[:chunk1_end]
    chunk2 = film[chunk1_end:chunk2_end]
    chunk3 = film[chunk2_end:]

    # Store the chunks for this file in a tuple
    chunks = (chunk1, chunk2, chunk3)

    # Append the tuple to the list of chunked files
    chunked_tokenized_films.append(chunks)

# Now chunked_tokenized_films contains the three chunks for each file
print(chunk3)

['ZED\n\n          (mulling it over)\n\n          Right-- right.', 'Then again, if the\n\n          rules say--\n\n          INANNA\n\n          Do you always follow the rules?', 'ZED\n\n          What is it with chicks always\n\n          wanting guys to break the rules?', 'You-all have this bad boy complex----\n\n          She kisses him fiercely on the lips.', 'INANNA\n\n          Do this for me.', "ZED\n\n          I'm going in.", 'He enters the Holy of Holies.', 'A FIGURE darts between the\n\n          columns and hides in the shadows.', 'A beam of moonlight\n\n          reveals the mark of Cain on his forehead.', 'CUT TO:\n\n          58                                                              INT.', 'HOLY OF HOLIES   - CONTINUOUS   58\n\n          Zed steps into the chamber and immediately prostrates\n\n          himself, face down on the floor, arms outstretched in\n\n          supplication.', 'He lies there with his eyes tightly shut,\n\n          expecting to be vaporized

In [34]:


def film_chunker(film):
    chunked_tokenized_films = []
    # Define the chunk sizes
    chunk1_size = 0.25
    chunk2_size = 0.50
    chunk3_size = 0.25
    for film in tokenized_films:
        # Calculate the sizes of each chunk based on the length of the file
        total_length = len(film)
        chunk1_end = int(total_length * chunk1_size)
        chunk2_end = int(total_length * (chunk1_size + chunk2_size))
    
        # Divide the file into three chunks
        chunk1 = film[:chunk1_end]
        chunk2 = film[chunk1_end:chunk2_end]
        chunk3 = film[chunk2_end:]
    
        # Store the chunks for this file in a tuple
        chunks = (chunk1, chunk2, chunk3)
    
        # Append the tuple to the list of chunked files
        # chunked_tokenized_films.append(chunks)
        return chunks

In [35]:
pieces = film_chunker(tokenized_films[0])

hhh

In [36]:
print(pieces[2][:20])

['of', 'him', 'she', "'s", 'in', 'baggy', 'climbing', 'in', 'snow', 'dungarees', 'or', 'that', 'existed', 'on', 'overalls', 'with', 'her', 'the', 'tape', '-']


In [37]:
film_chunks = []
for film in tokenized_films:
    chunked = film_chunker(film)
    film_chunks.append(chunked)


In [43]:
pieces = film_chunks[0]
print(pieces[0])

['127', 'HOURS', 'Written', 'by', 'Simon', 'Beaufoy', '&', 'Danny', 'Boyle', 'EXT', '.', 'CROWD', 'SCENES', '.', 'VARIOUS', '.', 'A', 'massive', 'crowd', ',', 'it', 'could', 'be', 'a', 'sports', 'stadium', ',', 'a', 'u2', 'farewell', 'show', 'or', 'new', 'year', "'s", 'eve', 'on', 'copacabana', 'beach', ',', 'but', 'whatever', 'it', 'is', 'there', 'are', 'thousands', 'and', 'thousands', 'of', 'us', '.', 'A', 'mexican', 'wave', 'erupts', 'success', ',', 'celebration', ',', 'with', 'so', 'many', 'involved', "it's", 'impossible', 'to', 'pick', 'out', 'anyone', 'individually', '.', 'Critical', 'mass', 'cyclists', ',', 'easter', 'crowds', 'at', 'st.', 'Peter', "'s", 'basilica', ',', 'nyc', 'marathon', ',', '4,000', 'flash', 'mobbers', 'doing', 'the', 'silent', 'disco', 'at', 'london', "'s", 'victoria', 'station', ',', 'india', "'s", 'kumbh', 'mela', ',', "macy's", 'thanksgiving', 'day', 'parade', ',', 'raves', ',', 'subway', 'parties', ',', 'the', 'daytona', '500.', '.', '.', '.', 'Graduall

In [52]:
unzipper = list(zip(*film_chunks))

In [61]:
# len(unzipper[0])
print(unzipper[0][0])
# type(unzipper[0])

['127', 'HOURS', 'Written', 'by', 'Simon', 'Beaufoy', '&', 'Danny', 'Boyle', 'EXT', '.', 'CROWD', 'SCENES', '.', 'VARIOUS', '.', 'A', 'massive', 'crowd', ',', 'it', 'could', 'be', 'a', 'sports', 'stadium', ',', 'a', 'u2', 'farewell', 'show', 'or', 'new', 'year', "'s", 'eve', 'on', 'copacabana', 'beach', ',', 'but', 'whatever', 'it', 'is', 'there', 'are', 'thousands', 'and', 'thousands', 'of', 'us', '.', 'A', 'mexican', 'wave', 'erupts', 'success', ',', 'celebration', ',', 'with', 'so', 'many', 'involved', "it's", 'impossible', 'to', 'pick', 'out', 'anyone', 'individually', '.', 'Critical', 'mass', 'cyclists', ',', 'easter', 'crowds', 'at', 'st.', 'Peter', "'s", 'basilica', ',', 'nyc', 'marathon', ',', '4,000', 'flash', 'mobbers', 'doing', 'the', 'silent', 'disco', 'at', 'london', "'s", 'victoria', 'station', ',', 'india', "'s", 'kumbh', 'mela', ',', "macy's", 'thanksgiving', 'day', 'parade', ',', 'raves', ',', 'subway', 'parties', ',', 'the', 'daytona', '500.', '.', '.', '.', 'Graduall